In [ ]:
import pickle
from cryptography.fernet import Fernet
import os
import time

def Singleton(cls):
    _instance = {}

    def _singleton(*args, **kwargs):
        if cls not in _instance:
            _instance[cls] = cls(*args, **kwargs)
        return _instance[cls]

    return _singleton

@Singleton
class A:
    a = 888

    def __init__(self, x=0):
        self.x = x
# Generate a key and save it to a file
def generate_key():
    key = Fernet.generate_key()
    with open("secret.key", "wb") as key_file:
        key_file.write(key)

# Load key from file
def load_key():
    return open("secret.key", "rb").read()

# Encrypt data
def encrypt(data, key):
    f = Fernet(key)
    return f.encrypt(data)

# Decrypt data
def decrypt(data, key):
    f = Fernet(key)
    return f.decrypt(data)

# Serialized objects 
def serialize_object(obj):
    return pickle.dumps(obj)

# Deserialized objects
def deserialize_object(data):
    return pickle.loads(data)

# Save the object to a file
def save_to_file(obj, filename, key):
    start_time = time.time()
    serialized_data = serialize_object(obj)
    serialization_time = time.time() - start_time

    start_time = time.time()
    encrypted_data = encrypt(serialized_data, key)
    encryption_time = time.time() - start_time

    start_time = time.time()
    with open(filename, 'wb') as f:
        f.write(encrypted_data)
    file_write_time = time.time() - start_time

    return serialization_time, encryption_time, file_write_time

# Load objects from files
def load_from_file(filename, key):
    start_time = time.time()
    with open(filename, 'rb') as f:
        encrypted_data = f.read()
    file_read_time = time.time() - start_time

    start_time = time.time()
    decrypted_data = decrypt(encrypted_data, key)
    decryption_time = time.time() - start_time

    start_time = time.time()
    obj = deserialize_object(decrypted_data)
    deserialization_time = time.time() - start_time

    return obj, file_read_time, decryption_time, deserialization_time

# Generate and load keys
if not os.path.exists("secret.key"):
    generate_key()
key = load_key()

# Scene simulation
def simulate():
    # Before running the program
    a1 = A(888)
    print("Before shutdown:", a1)

    # Save state before closing simulation program
    serialization_time, encryption_time, file_write_time = save_to_file(a1, 'singleton.pkl', key)
    print(f"Serialization time: {serialization_time:.6f}s, Encryption time: {encryption_time:.6f}s, File write time: {file_write_time:.6f}s")

    # Clear instances to simulate program shutdown
    A._instance = {}

    # Loading status at program startup
    loaded_a1, file_read_time, decryption_time, deserialization_time = load_from_file('singleton.pkl', key)
    A._instance[A] = loaded_a1  # Restore singleton reference

    print("After startup:", loaded_a1)
    print(f"File read time: {file_read_time:.6f}s, Decryption time: {decryption_time:.6f}s, Deserialization time: {deserialization_time:.6f}s")



simulate()
